In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from fonte_dados.fabrica import FabricaFonteDados
from treinamento.coerencia import CalculadorCoerencia
from util import constants

fabrica = FabricaFonteDados()

# Análise de coerência de quantidade de tópicos

In [3]:
fonte_dados_origem = fabrica.get_fonte_dados(constants.NERDS_VIAJANTES)
fonte_dados_origem.carregar_dados()

In [7]:
topicos_from = 5
topicos_to = 120

In [ ]:
calculador_coerencia = CalculadorCoerencia(fonte_dados_origem)
coerencias = calculador_coerencia.calcular_coerencias(topicos_from, topicos_to, debug=True)
coerencias.to_csv(f'/home/helder/estudos/tcc-pucmg-2/src/python/notebooks/treinamento/coerencias_{topicos_from}-to-{topicos_to}.csv')